In [3]:
from typing import Any, Dict, Iterator, List, Optional

from langchain_core.callbacks import (
    CallbackManagerForLLMRun,
)
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    HumanMessage
)
from langchain_core.messages.ai import UsageMetadata
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from pydantic import Field

## CustomLLM

In [2]:
class ChatParrotLink(BaseChatModel):
    
    model_name: str = Field(alias="model")
    """The name of the model"""
    parrot_buffer_length: int
    """The number of characters from the last message of the prompt to be echoed."""
    temperature: Optional[float] = None
    max_tokens: Optional[int] = None
    timeout: Optional[int] = None
    stop: Optional[List[str]] = None
    max_retries: int = 2

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        last_message = messages[-1]
        print(type(last_message))
        tokens = last_message.content[: self.parrot_buffer_length]
        ct_input_tokens = sum(len(message.content) for message in messages)
        ct_output_tokens = len(tokens)
        message = AIMessage(
            content=tokens,
            additional_kwargs={},  # Used to add additional payload to the message
            response_metadata={  # Use for response metadata
                "time_in_seconds": 3,
                "model_name": self.model_name,
            },
            usage_metadata={
                "input_tokens": ct_input_tokens,
                "output_tokens": ct_output_tokens,
                "total_tokens": ct_input_tokens + ct_output_tokens,
            },
        )
        ##

        generation = ChatGeneration(message=message)
        return ChatResult(generations=[generation])

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        last_message = messages[-1]
        tokens = str(last_message.content[: self.parrot_buffer_length])
        ct_input_tokens = sum(len(message.content) for message in messages)

        for token in tokens:
            usage_metadata = UsageMetadata(
                {
                    "input_tokens": ct_input_tokens,
                    "output_tokens": 1,
                    "total_tokens": ct_input_tokens + 1,
                }
            )
            ct_input_tokens = 0
            chunk = ChatGenerationChunk(
                message=AIMessageChunk(content=token, usage_metadata=usage_metadata)
            )

            if run_manager:
                # This is optional in newer versions of LangChain
                # The on_llm_new_token will be called automatically
                run_manager.on_llm_new_token(token, chunk=chunk)

            yield chunk

        # Let's add some other information (e.g., response metadata)
        chunk = ChatGenerationChunk(
            message=AIMessageChunk(
                content="",
                response_metadata={"time_in_sec": 3, "model_name": self.model_name},
            )
        )
        if run_manager:
            # This is optional in newer versions of LangChain
            # The on_llm_new_token will be called automatically
            run_manager.on_llm_new_token(token, chunk=chunk)
        yield chunk

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model."""
        return "echoing-chat-model-advanced"

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        This information is used by the LangChain callback system, which
        is used for tracing purposes make it possible to monitor LLMs.
        """
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": self.model_name,
        }

In [4]:
model = ChatParrotLink(parrot_buffer_length=3, model="my_custom_model")

model.invoke(
    [
        HumanMessage(content="hello!"),
        AIMessage(content="Hi there human!"),
        HumanMessage(content="Meow!"),
    ]
)

<class 'langchain_core.messages.human.HumanMessage'>


AIMessage(content='Meo', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-eff55b08-f020-4679-bb7c-5c5ef230c6ad-0', usage_metadata={'input_tokens': 26, 'output_tokens': 3, 'total_tokens': 29})

In [5]:
model.invoke("hello")

<class 'langchain_core.messages.human.HumanMessage'>


AIMessage(content='hel', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-c74bc94e-fd6c-4336-b7a9-9d77a8aba583-0', usage_metadata={'input_tokens': 5, 'output_tokens': 3, 'total_tokens': 8})

In [6]:
model.batch(["hello", "goodbye"])

<class 'langchain_core.messages.human.HumanMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


[AIMessage(content='hel', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-b9de8e80-4c0b-469c-ab8a-ffa5dd5f9f20-0', usage_metadata={'input_tokens': 5, 'output_tokens': 3, 'total_tokens': 8}),
 AIMessage(content='goo', additional_kwargs={}, response_metadata={'time_in_seconds': 3, 'model_name': 'my_custom_model'}, id='run-b470ea1b-20e3-4941-916e-2b30f9a19afc-0', usage_metadata={'input_tokens': 7, 'output_tokens': 3, 'total_tokens': 10})]

In [7]:
for chunk in model.stream("horse"):
    print(chunk.content, end="|")

h|o|r||

In [8]:
async for chunk in model.astream("cat"):
    print(chunk.content, end="|")

c|a|t||

In [9]:
async for event in model.astream_events("cat", version="v1"):
    print(event)

{'event': 'on_chat_model_start', 'run_id': '9388bf2d-f17b-4512-856c-5bb62878b44d', 'name': 'ChatParrotLink', 'tags': [], 'metadata': {}, 'data': {'input': 'cat'}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '9388bf2d-f17b-4512-856c-5bb62878b44d', 'tags': [], 'metadata': {}, 'name': 'ChatParrotLink', 'data': {'chunk': AIMessageChunk(content='c', additional_kwargs={}, response_metadata={}, id='run-9388bf2d-f17b-4512-856c-5bb62878b44d', usage_metadata={'input_tokens': 3, 'output_tokens': 1, 'total_tokens': 4})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '9388bf2d-f17b-4512-856c-5bb62878b44d', 'tags': [], 'metadata': {}, 'name': 'ChatParrotLink', 'data': {'chunk': AIMessageChunk(content='a', additional_kwargs={}, response_metadata={}, id='run-9388bf2d-f17b-4512-856c-5bb62878b44d', usage_metadata={'input_tokens': 0, 'output_tokens': 1, 'total_tokens': 1})}, 'parent_ids': []}
{'event': 'on_chat_model_stream', 'run_id': '9388bf2d-f17b-4512-856c-5bb62878b4

In [10]:
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()
tool = TavilySearchResults(max_results = 3)
tools = [tool]

# llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = model.bind_tools(tools)

NotImplementedError: 

In [12]:
tool.model_dump()

{'name': 'tavily_search_results_json',
 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
 'args_schema': langchain_community.tools.tavily_search.tool.TavilyInput,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content_and_artifact',
 'max_results': 3,
 'search_depth': 'advanced',
 'include_domains': [],
 'exclude_domains': [],
 'include_answer': False,
 'include_raw_content': False,
 'include_images': False,
 'api_wrapper': {'tavily_api_key': SecretStr('**********')}}

In [1]:
from typing import Annotated
from langchain.tools import tool


def add_numbers2(a: Annotated[int, "first number to add"], b: Annotated[int, "second number to add"]) -> int:
    """Add two numbers"""
    return a + b

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [22]:
add_numbers2.model_dump()

{'name': 'add_numbers2',
 'description': 'Add two numbers',
 'args_schema': langchain_core.utils.pydantic.add_numbers2,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': <function __main__.add_numbers2(a: typing.Annotated[int, 'first number to add'], b: typing.Annotated[int, 'second number to add']) -> int>,
 'coroutine': None}

In [23]:
add_numbers.model_dump()

{'name': 'add_numbers',
 'description': 'Add two numbers',
 'args_schema': langchain_core.utils.pydantic.add_numbers,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': <function __main__.add_numbers(a: int, b: int) -> int>,
 'coroutine': None}

In [2]:
add_numbers2(2,3)

5